In [11]:
#ライブラリインポート
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import time
from tqdm.notebook import tqdm
import pandas as pd

1:札幌　2:函館　3:福島　4:新潟　5:東京　6:中山　7:中京　8:京都　9:阪神　10:小倉

In [12]:
#レースIDのリスト作成
#URL構造　"https://db.netkeiba.com/race/"　+　race_id
#race_id構造　年,開催場所,回数,何日目か,レース

race_id_list = []
for year in range(2008,2021):
    #(2008,2021)
    for place in range(1, 11):
        #(1, 11)
        for kai in range(1, 7):
            #(1,7)
            for day in range(1, 13):
                #(1,13)
                for r in range(1, 13):
                    #(1,13)
                    race_id = str(year).zfill(4) + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                    race_id_list.append(race_id)


In [13]:
#レース情報についてスクレイピングする関数定義

def scrape_race_info(race_id_list):
    race_infos = {}
    race_class_list = []
    for race_id in tqdm(race_id_list):
        try:
            #レース結果ページから、レースの情報について収集し、整形する
            html = urlopen('https://db.netkeiba.com/race/' + race_id)
            bs = BeautifulSoup(html.read(),'html.parser')

            #レース情報についてスクレイピング
            #[レース名、距離、天気、馬場、馬場状態、日付]
            texts = (
                        bs.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                        + bs.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                        )
            race_name = bs.find("div", attrs={"class": "data_intro"}).find_all("h1")[0].text
            #infoに正規表現を使ってtextsを取得
            info = re.findall(r"\w+", texts)
            
            #目野追加
            race_class = bs.find(class_="smalltxt").text.replace(u'\xa0', u' ').split(' ')[2]
            race_class_list.append(race_class)

            #整形
            info_dict = {}
            for text in info:
                info_dict["race_name"] = race_name
                
                #目野追加
                info_dict["race_class"] = race_class
                
                if "m" in text:
                    info_dict["course_len"] = int(re.findall(r"\d+", text)[0])
                if "m" in text:
                    info_dict["rotation"] = str(re.findall(r"[右,左]", text)[0])
                if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                    info_dict["weather"] = text
                if text in ["芝", "ダート"]:
                    info_dict["race_type"] = text
                if text in ["良", "稍重", "重", "不良"]:
                    info_dict["ground_state"] = text
                if "障" in text:
                    info_dict["race_type"] = "障害"
                if "年" in text:
                    info_dict["date"] = text
        
            race_infos[race_id] = info_dict
            #time.sleep(1)
        except IndexError:
            continue
        except AttributeError:
            continue
        except Exception as e:
            print(e)
            break
        except:
            break
    
    #目野追加、race_classの一覧とclassごとのレース回数をrace_classes.csvに出力
    race_class_list_unduplicated = sorted(list(set(race_class_list)))
    race_class_info_list = []
    for v in race_class_list_unduplicated:
        race_class_info = {}
        race_class_info["race_class_name"] = v
        race_class_info["count"] = race_class_list.count(v)
        race_class_info_list.append(race_class_info)
    race_class_df = pd.DataFrame(race_class_info_list)
    race_class_df.to_csv("race_classes.csv")
    
    return race_infos

In [ ]:
#関数実行
start = time.perf_counter()
race_infos=scrape_race_info(race_id_list)
print("処理時間 = "+str(time.perf_counter() - start))
print(race_infos)

  0%|          | 0/112320 [00:00<?, ?it/s]

In [5]:
#DataFrame型に変更
race_infos = pd.DataFrame(race_infos)
race_infos_T = race_infos.T
race_infos_all = race_infos_T.rename_axis('race_id').reset_index()

#df表示
race_infos_all


,race_id,race_name,race_class,course_len,rotation,weather,race_type,ground_state,date
0,202101010101,3歳未勝利,3歳未勝利,1200,右,小雨,芝,良,2021年6月12日
1,202101010102,3歳未勝利,3歳未勝利,1700,右,曇,ダート,良,2021年6月12日
2,202101010103,3歳未勝利,3歳未勝利,1000,右,曇,ダート,良,2021年6月12日
3,202101010104,3歳未勝利,3歳未勝利,2000,右,曇,芝,良,2021年6月12日
4,202101010105,2歳新馬,2歳新馬,1000,右,曇,芝,良,2021年6月12日
...,...,...,...,...,...,...,...,...,...
2958,202110040808,3歳以上1勝クラス,3歳以上1勝クラス,1000,右,曇,ダート,稍重,2021年9月5日
2959,202110040809,筑後川特別,3歳以上2勝クラス,1800,右,小雨,芝,良,2021年9月5日
2960,202110040810,西日本スポーツ杯,3歳以上3勝クラス,1700,右,小雨,ダート,稍重,2021年9月5日
2961,202110040811,第41回小倉2歳ステークス(G3),2歳オープン,1200,右,小雨,芝,良,2021年9月5日


In [6]:

#CSV形式で保存
race_infos_all.to_csv("race_infos.csv",encoding="shift_jis")